# QuPath Cell Masks

In this example we create cell segmentations in [QuPath](https://qupath.github.io/), convert them to a binary mask in python, and push both the original image and and cell mask to ManiVault.

We use the `OS-2.ndpi` from the [OpenSlide example images](https://openslide.cs.cmu.edu/download/openslide-testdata/Hamamatsu/) as also used in the [QuPath cell detection tutorial](https://qupath.readthedocs.io/en/0.5/docs/tutorials/cell_detection.html). After downloading, the file will automatically be called `OS-2.tiff`.

First, we create a cell mask with QuPath

1. Open the `OS-2.tiff` file in QuPath
2. Create a rectangular annotation (ROI) and export it as on ome.tiff
3. Open this new `OS2-ROI.ome.tif` in QuPath
4. Segment the cells via `Analyze` > `Cell detection`. First, create a rectangular annocation that covers the entire image, and then ,in the pop-up ,de-select the options "Include cell nucleus" and "Make measurements". Click "Run".
6. Export the cell masks to GeoJSON: Select all cell annotations, click `File` > `Export Objects as GeoJSON`, select "Pretty JSON" and "Exclude measurements" and click "OK".


OS-2.ndpi in QuPath | ROI with segmeted cells
- | -
![Full cell image](full_image.png) | ![Cell mask](segmented_cells.png)

Let's define some helper functions:

In [ ]:
def load_json(load_json_file_path):
    import json
    with open(load_json_file_path) as load_json_file:
        loaded_json_file = json.load(load_json_file)
    return loaded_json_file

def load_tiff(load_tiff_path):
    from tifffile import tifffile
    return tifffile.imread(load_tiff_path)


def extract_mask_from_geojson(geo_json_in, mask_out_shape = (1024, 1024)):
    """
    :param geo_json_in: json file
    :param mask_out_shape: optional, Define output raster shape (e.g., 512x512) and transform (identity for simple pixel grid)
    :return:
    """
    from shapely.geometry import shape
    from rasterio.features import rasterize

    # Extract polygon geometries
    shapes = [shape(feature['geometry']) for feature in geo_json_in['features']]

    # Rasterize polygons into mask
    mask_out = rasterize(
        [(geom, 1) for geom in shapes],
        out_shape=mask_out_shape,
        fill=0,
        default_value=1
    )

    return mask_out

def plot_mask(mask_plot):
    import matplotlib.pyplot as plt
    plt.imshow(mask_plot, cmap='gray')
    plt.axis('off')
    plt.show()

def plot_image(image_plot):
    import matplotlib.pyplot as plt
    plt.imshow(image_plot)
    plt.axis('off')
    plt.show()

def plot_image_with_mask(image_plot, mask_plot):
    import matplotlib.pyplot as plt
    import numpy as np

    # Show the image
    fig, ax = plt.subplots()
    plt.imshow(image_plot)

    # Create an RGBA version of the mask
    mask_rgba = np.zeros((mask_plot.shape[0], mask_plot.shape[1], 4))

    # Set color (e.g., red) for mask areas
    mask_rgba[..., 0] = 1.0         # Red channel
    mask_rgba[..., 3] = mask_plot   # Alpha channel — 0 if mask=0, 1 if mask=1

    # Overlay the mask
    ax.imshow(mask_rgba)

    plt.axis('off')
    plt.show()


And now we can load the image and cell segmentation file:

In [ ]:
# Define source locations
path_image = "./examples/JupyterPlugin/cell_segmentation_qupath/OS2-ROI.ome.tif"
path_json = "./examples/JupyterPlugin/cell_segmentation_qupath/OS2-ROI.geojson"

# Load data
image = load_tiff(path_image)
j = load_json(path_json)

# Convert QuPath segmentation to mask
mask = extract_mask_from_geojson(j, (image.shape[0], image.shape[1]))

# Plot results
plot_mask(mask)
plot_image(image)
plot_image_with_mask(image, mask)

Let's push the image and mask to ManiVault:

In [ ]:
# Connect to ManiVault
import mvstudio.data
mv = mvstudio.data.Hierarchy()

# Push image and mask
image_mv_ref = mv.addImageItem(image, "Cell Image")
mask_mv_ref = mv.addImageItem(mask, "Mask Image")

Now we want to push a selection based on the mask to ManiVault:

In [ ]:
import numpy as np

# The internal image indexing in ManiVault is flipped compared to numpy's data model
mask_selection = np.flatnonzero(np.flipud(mask) > 0)

image_mv_ref.setSelection(mask_selection)
mask_mv_ref.setSelection(mask_selection)

We can now visualize the image and mask-selection in ManiVault:

Image and mask-selection in ManiVault | Selected subset of the cell data
- | -
![Image and mask-selection in ManiVault](image_selection_manivault.png) | ![Subset image](image_subset_manivault.png)

We can now interactively select pixels in ManiVault and ask for the selected indices for further analysis on the Python side:

In [ ]:
manual_selection = image_mv_ref.getSelection()
display(manual_selection)